# Aggregating trajectories

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=1-tutorials/9-aggregating-trajectories.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/movingpandas/movingpandas-examples/blob/main/1-tutorials/9-aggregating-trajectories.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://movingpandas.github.io/movingpandas-website/1-tutorials/9-aggregating-trajectories.html)

The aggregation approach implemented in TrajectoryCollectionAggregator is based on Andrienko, N., & Andrienko, G. (2011). Spatial generalization and aggregation of massive movement data. IEEE Transactions on visualization and computer graphics, 17(2), 205-219. and consists of the following main steps:

1. Extracting characteristic points from the trajectories
2. Grouping the extracted points by spatial proximity
3. Computing group centroids and corresponding Voronoi cells
4. Dividing trajectories into segments according to the Voronoi cells
5. Counting transitions from one cell to another

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas
import matplotlib.pyplot as plt
import folium

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim

import warnings

warnings.filterwarnings("ignore")

plot_defaults = {"linewidth": 5, "capstyle": "round", "figsize": (9, 3), "legend": True}
opts.defaults(
    opts.Overlay(active_tools=["wheel_zoom"], frame_width=500, frame_height=400)
)

mpd.show_versions()

In [ ]:
gdf = read_file("../data/geolife_small.gpkg")
tc = mpd.TrajectoryCollection(gdf, "trajectory_id", t="t")

In [ ]:
tc.hvplot(line_width=7.0, tiles="StamenTonerBackground")

In [ ]:
tc.explore(column="trajectory_id", cmap="plasma", style_kwds={"weight": 4})

## TrajectoryCollectionAggregator

Generalizing the trip trajectories significantly speeds up the following aggregation step.

In [ ]:
generalized = mpd.MinDistanceGeneralizer(tc).generalize(tolerance=100)

In [ ]:
aggregator = mpd.TrajectoryCollectionAggregator(
    generalized,
    max_distance=1000,
    min_distance=100,
    min_stop_duration=timedelta(minutes=5),
)

In [ ]:
pts = aggregator.get_significant_points_gdf()
clusters = aggregator.get_clusters_gdf()
(
    pts.hvplot(geo=True, tiles="StamenTonerBackground")
    * clusters.hvplot(geo=True, color="red")
)

In [ ]:
m = pts.explore(marker_kwds={"radius": 3}, name="Significant points")

clusters.explore(m=m, color="red", marker_kwds={"radius": 3}, name="Cluster centroids")

folium.TileLayer("CartoDB positron").add_to(m)
folium.LayerControl().add_to(m)

m

In [ ]:
flows = aggregator.get_flows_gdf()

In [ ]:
(
    flows.hvplot(
        geo=True,
        hover_cols=["weight"],
        line_width=dim("weight") * 7,
        color="#1f77b3",
        tiles="StamenTonerBackground",
    )
    * clusters.hvplot(geo=True, color="red", size=dim("n"))
)

In [ ]:
m = flows.explore(
    style_kwds={"weight": 5},
    name="Flows",
)

clusters.explore(
    m=m,
    color="red",
    style_kwds={"style_function": lambda x: {"radius": x["properties"]["n"]}},
    name="Clusters",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

m